In [1]:
!pip install qiskit

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 12.2 MB/s eta 0:00:00
  Created wheel fo

In [2]:
!pip install git+https://github.com/FilipeChagasDev/dobslit-senai-lib-a3.git

  Cloning https://github.com/FilipeChagasDev/dobslit-senai-lib-a3.git to /tmp/pip-req-build-dwsh79c_
  Running command git clone --filter=blob:none --quiet https://github.com/FilipeChagasDev/dobslit-senai-lib-a3.git /tmp/pip-req-build-dwsh79c_
  Resolved https://github.com/FilipeChagasDev/dobslit-senai-lib-a3.git to commit d341ecb90bf61c46477f9d6529599843ed1e709f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dobslit-senai-a3: filename=dobslit_senai_a3-0.0.0-py3-none-any.whl size=4080 sha256=b00585a5bba7d74c9e778ef1d71b384625b66e0aa88545e97c1f5a4f356e1b9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-wrpt8m6k/wheels/a6/ca/20/8d93f11d34c5e5a5afea407071d8f06e1d860b641110e1a8b3
Successfully built dobslit-senai-a3


Implementar o algoritmo de Grover para encontrar a solução da expressão booleana - Y = not ((x1 or not x3 or x4) and (not x2 or x3 or not x4))


In [4]:
from dobslit_senai_a3 import *


In [6]:
class MySat(GroverAlgorithm):
  def prepare(self): # Alocação de Qubits
    # Cria um Qubit para cada variavel
    self.x1 = self.create_qubit('x1')
    self.x2 = self.create_qubit('x2')
    self.x3 = self.create_qubit('x3')
    self.x4 = self.create_qubit('x4')

    # Cria qubits auxiliares
    self.aux1 = self.create_qubit('aux1')
    self.aux2 = self.create_qubit('aux2')

    # Cria qubit de resultado
    self.y = self.create_qubit('y')

  def build_search_space(self):
    self.h(self.x1)
    self.h(self.x2)
    self.h(self.x3)
    self.h(self.x4)

    # aux1 = x1 or not x3 or x4
    self.logic_not(self.x3)

    self.logic_or([self.x1, self.x3, self.x4], self.aux1)

    self.logic_not(self.x3)

    # aux2 = not x2 or x3 or not x4
    self.logic_not(self.x2)
    self.logic_not(self.x4)

    self.logic_or([self.x2, self.x3, self.x4], self.aux2)

    self.logic_not(self.x2)
    self.logic_not(self.x4)

    # y = not(aux1 and aux2)

    self.logic_and([self.aux1, self.aux2], self.y)
    self.logic_not(self.y)

  def revert_search_space(self):
    self.logic_not(self.y)
    self.logic_and([self.aux1, self.aux2], self.y)

    self.logic_not(self.x4)
    self.logic_not(self.x2)
    self.logic_or_dg([self.x2, self.x3, self.x4], self.aux2)
    self.logic_not(self.x4)
    self.logic_not(self.x2)

    self.logic_not(self.x3)
    # dg = dagger, inverso do operador or, pois ele não é simétrico
    self.logic_or_dg([self.x1, self.x3, self.x4], self.aux1)
    self.logic_not(self.x3)

    self.h(self.x4)
    self.h(self.x3)
    self.h(self.x2)
    self.h(self.x1)



In [17]:
mysat = MySat()

# A quantidade de iterações do algoritmo de grover para esse caso é 2, outro número diminui a probabilidade de obtenção dos resultados
mysat.build_all(mysat.y, 2)

mysat.circ.draw()

┌───┐ ░                                                        »
phase_ancilla: ┤ H ├─░────────────────────────────────────────────────────────»
               └───┘ ░ ┌───┐┌───┐          ┌───┐                              »
         q_x1: ──────░─┤ H ├┤ X ├───────■──┤ X ├──────────────────────────────»
                     ░ ├───┤├───┤┌───┐  │  └───┘               ┌───┐┌───┐     »
         q_x2: ──────░─┤ H ├┤ X ├┤ X ├──┼───────────────────■──┤ X ├┤ X ├─────»
                     ░ ├───┤├───┤├───┤  │  ┌───┐┌───┐┌───┐  │  ├───┤└───┘     »
         q_x3: ──────░─┤ H ├┤ X ├┤ X ├──■──┤ X ├┤ X ├┤ X ├──■──┤ X ├──────────»
                     ░ ├───┤├───┤└───┘  │  ├───┤├───┤├───┤  │  ├───┤┌───┐     »
         q_x4: ──────░─┤ H ├┤ X ├───────■──┤ X ├┤ X ├┤ X ├──■──┤ X ├┤ X ├─────»
                     ░ └───┘└───┘     ┌─┴─┐├───┤└───┘└───┘  │  └───┘└───┘     »
       q_aux1: ──────░────────────────┤ X ├┤ X ├────────────┼─────────■───────»
                     ░                └───┘└───┘          ┌─┴─┐┌───┐  │       »
       q_aux2: ──────░────────────────────────────────────┤ X ├┤ X ├──■───────»
                     ░                                    └───┘└───┘┌─┴─┐┌───┐»
          q_y: ──────░──────────────────────────────────────────────┤ X ├┤ X ├»
                     ░                                              └───┘└───┘»
       c_x1: 1/═══════════════════════════════════════════════════════════════»
                                                                              »
       c_x2: 1/═══════════════════════════════════════════════════════════════»
                                                                              »
       c_x3: 1/═══════════════════════════════════════════════════════════════»
                                                                              »
       c_x4: 1/═══════════════════════════════════════════════════════════════»
                                                                              »
     c_aux1: 1/═══════════════════════════════════════════════════════════════»
                                                                              »
     c_aux2: 1/═══════════════════════════════════════════════════════════════»
                                                                              »
        c_y: 1/═══════════════════════════════════════════════════════════════»
                                                                              »
«                ░     ░                                                   »
«phase_ancilla: ─░──■──░───────────────────────────────────────────────────»
«                ░  │  ░ ┌───┐                                   ┌───┐┌───┐»
«         q_x1: ─░──┼──░─┤ X ├────────────────────────────────■──┤ X ├┤ H ├»
«                ░  │  ░ ├───┤┌───┐          ┌───┐┌───┐┌───┐  │  └───┘└───┘»
«         q_x2: ─░──┼──░─┤ X ├┤ X ├───────■──┤ X ├┤ X ├┤ H ├──┼────────────»
«                ░  │  ░ ├───┤└───┘       │  ├───┤├───┤├───┤  │  ┌───┐┌───┐»
«         q_x3: ─░──┼──░─┤ X ├────────────■──┤ X ├┤ X ├┤ X ├──■──┤ X ├┤ X ├»
«                ░  │  ░ ├───┤┌───┐       │  ├───┤├───┤├───┤  │  ├───┤├───┤»
«         q_x4: ─░──┼──░─┤ X ├┤ X ├───────■──┤ X ├┤ X ├┤ X ├──■──┤ X ├┤ H ├»
«                ░  │  ░ └───┘└───┘┌───┐  │  └───┘└───┘└───┘┌─┴─┐└───┘└───┘»
«       q_aux1: ─░──┼──░────────■──┤ X ├──┼─────────────────┤ X ├──────────»
«                ░  │  ░        │  ├───┤┌─┴─┐               └───┘          »
«       q_aux2: ─░──┼──░────────■──┤ X ├┤ X ├──────────────────────────────»
«                ░  │  ░ ┌───┐┌─┴─┐└───┘└───┘                              »
«          q_y: ─░──■──░─┤ X ├┤ X ├────────────────────────────────────────»
«                ░     ░ └───┘└───┘                                        »
«       c_x1: 1/═══════════════════════════════════════════════════════════»
«                                                                          »
«       c_x2: 1/════════════════════════════════════════════════════════

In [15]:
mysat.simulate()

,x1,x2,x3,x4,aux1,aux2,y,$freq
0,0,1,0,1,1,0,1,117
1,0,0,1,0,0,1,1,114
2,0,1,1,0,0,1,1,108
3,1,1,0,1,1,0,1,96
4,0,0,1,1,1,1,0,60
5,1,0,1,0,1,1,0,57
6,0,0,0,0,1,1,0,57
7,0,1,0,0,1,1,0,53
8,1,1,0,0,1,1,0,52
9,1,1,1,1,1,1,0,51


Exercicio - Y = (not x1 and x2 and not x3) or (x1 and not x2 and x3)

In [22]:
class MySatExercicio(GroverAlgorithm):
  def prepare(self):
    self.x1 = self.create_qubit('x1')
    self.x2 = self.create_qubit('x2')
    self.x3 = self.create_qubit('x3')

    self.aux1 = self.create_qubit('aux1')
    self.aux2 = self.create_qubit('aux2')

    self.y = self.create_qubit('y')

  def build_search_space(self):
    self.h(self.x1)
    self.h(self.x2)
    self.h(self.x3)

    # aux1 = not x1 and x2 and not x3
    self.logic_not(self.x1)
    self.logic_not(self.x3)

    self.logic_and([self.x1, self.x2, self.x3], self.aux1)

    self.logic_not(self.x1)
    self.logic_not(self.x3)

    # aux2 = x1 and not x2 and x3
    self.logic_not(self.x2)

    self.logic_and([self.x1, self.x2, self.x3], self.aux2)

    self.logic_not(self.x2)

    # y = aux1 or aux2
    self.logic_or([self.aux1, self.aux2], self.y)

  def revert_search_space(self):
    self.logic_or_dg([self.aux1, self.aux2], self.y)

    self.logic_not(self.x2)
    self.logic_and([self.x1, self.x2, self.x3], self.aux2)
    self.logic_not(self.x2)

    self.logic_not(self.x1)
    self.logic_not(self.x3)
    self.logic_and([self.x1, self.x2, self.x3], self.aux1)
    self.logic_not(self.x1)
    self.logic_not(self.x3)

    self.h(self.x1)
    self.h(self.x2)
    self.h(self.x3)

In [30]:
my_sat_exercicio = MySatExercicio()

# A quantidade de iterações do algoritmo de grover para esse caso é 2, outro número diminui a probabilidade de obtenção dos resultados
my_sat_exercicio.build_all(my_sat_exercicio.y, 2)

my_sat_exercicio.circ.draw()

┌───┐ ░                                          ░     ░      »
phase_ancilla: ┤ H ├─░──────────────────────────────────────────░──■──░──────»
               └───┘ ░ ┌───┐┌───┐     ┌───┐                     ░  │  ░      »
         q_x1: ──────░─┤ H ├┤ X ├──■──┤ X ├──■──────────────────░──┼──░──────»
                     ░ ├───┤└───┘  │  ├───┤  │  ┌───┐           ░  │  ░ ┌───┐»
         q_x2: ──────░─┤ H ├───────■──┤ X ├──■──┤ X ├───────────░──┼──░─┤ X ├»
                     ░ ├───┤┌───┐  │  ├───┤  │  └───┘           ░  │  ░ └───┘»
         q_x3: ──────░─┤ H ├┤ X ├──■──┤ X ├──■──────────────────░──┼──░──────»
                     ░ └───┘└───┘┌─┴─┐├───┤  │            ┌───┐ ░  │  ░ ┌───┐»
       q_aux1: ──────░───────────┤ X ├┤ X ├──┼─────────■──┤ X ├─░──┼──░─┤ X ├»
                     ░           └───┘└───┘┌─┴─┐┌───┐  │  ├───┤ ░  │  ░ ├───┤»
       q_aux2: ──────░─────────────────────┤ X ├┤ X ├──■──┤ X ├─░──┼──░─┤ X ├»
                     ░                     └───┘└───┘┌─┴─┐├───┤ ░  │  ░ ├───┤»
          q_y: ──────░───────────────────────────────┤ X ├┤ X ├─░──■──░─┤ X ├»
                     ░                               └───┘└───┘ ░     ░ └───┘»
       c_x1: 1/══════════════════════════════════════════════════════════════»
                                                                             »
       c_x2: 1/══════════════════════════════════════════════════════════════»
                                                                             »
       c_x3: 1/══════════════════════════════════════════════════════════════»
                                                                             »
     c_aux1: 1/══════════════════════════════════════════════════════════════»
                                                                             »
     c_aux2: 1/══════════════════════════════════════════════════════════════»
                                                                             »
        c_y: 1/══════════════════════════════════════════════════════════════»
                                                                             »
«                                                   ░ ┌───┐┌───┐┌───┐ ░      »
«phase_ancilla: ────────────────────────────────────░─┤ Z ├┤ X ├┤ Z ├─░──────»
«                              ┌───┐     ┌───┐┌───┐ ░ ├───┤└─┬─┘├───┤ ░ ┌───┐»
«         q_x1: ────────────■──┤ X ├──■──┤ X ├┤ H ├─░─┤ X ├──■──┤ X ├─░─┤ H ├»
«                           │  ├───┤  │  ├───┤└───┘ ░ ├───┤  │  ├───┤ ░ ├───┤»
«         q_x2: ────────────■──┤ X ├──■──┤ H ├──────░─┤ X ├──■──┤ X ├─░─┤ H ├»
«                           │  ├───┤  │  ├───┤┌───┐ ░ ├───┤  │  ├───┤ ░ ├───┤»
«         q_x3: ────────────■──┤ X ├──■──┤ X ├┤ H ├─░─┤ X ├──■──┤ X ├─░─┤ H ├»
«                    ┌───┐  │  └───┘┌─┴─┐└───┘└───┘ ░ ├───┤  │  ├───┤ ░ └───┘»
«       q_aux1: ──■──┤ X ├──┼───────┤ X ├───────────░─┤ X ├──■──┤ X ├─░──────»
«                 │  ├───┤┌─┴─┐     └───┘           ░ ├───┤  │  ├───┤ ░      »
«       q_aux2: ──■──┤ X ├┤ X ├─────────────────────░─┤ X ├──■──┤ X ├─░──────»
«               ┌─┴─┐└───┘└───┘                     ░ ├───┤  │  ├───┤ ░      »
«          q_y: ┤ X ├───────────────────────────────░─┤ X ├──■──┤ X ├─░──────»
«               └───┘                               ░ └───┘     └───┘ ░      »
«       c_x1: 1/═════════════════════════════════════════════════════════════»
«                                                                            »
«       c_x2: 1/═════════════════════════════════════════════════════════════»
«                                                                            »
«       c_x3: 1/═════════════════════════════════════════════════════════════»
«                                                                            »
«     c_aux1: 1/═════════════════════════════════════════════════════════════»
«                                                                            »
«     c_aux2: 1/═════════════════════════════════════════════════

In [31]:
my_sat_exercicio.simulate()

,x1,x2,x3,aux1,aux2,y,$freq
0,0,1,0,1,0,1,515
1,1,0,1,0,1,1,470
2,1,1,0,0,0,0,10
3,1,1,1,0,0,0,9
4,0,0,1,0,0,0,6
5,0,1,1,0,0,0,5
6,0,0,0,0,0,0,5
7,1,0,0,0,0,0,4
